In [1]:
#Import Libaries
import os
import numpy as np 
#from random import shuffle
from sklearn.utils import shuffle
import matplotlib
import matplotlib.pyplot as plt 

import h5py
%matplotlib inline
import matplotlib.pyplot as plt
# --- Signal Processing Libary
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy.signal import butter, lfilter, freqz, hilbert, filtfilt, convolve 
import scipy 
import IPython.display as ipd
# --- Machine Learining
import keras
from keras.models import Model
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from keras.models import load_model
from keras.models import model_from_json
from sklearn.model_selection import train_test_split,cross_val_score

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from keras.layers import Dense, Input, Flatten, MaxPooling2D
from keras.models import Model
from keras.layers import Conv1D,Conv2D,Dropout,Reshape,Activation
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.models import load_model
from sklearn.metrics import mean_squared_error

$m(f_m) = \displaystyle\frac{\displaystyle\int_{0}^{\infty} h^2(t)e^{-j2\pi{f_m}t}dt}{\displaystyle\int_{0}^{\infty} h^2(t)dt}$

$\displaystyle m(f_m)=\frac{1}{{\left[ 1+\left(2\pi f_m \frac{T_R}{13.8}\right)^2 \right]}^{{\frac{1}{2}}}},$

$\displaystyle \frac{1}{m(f_m)^2}=1+\left(2\pi f_m \frac{T_R}{13.8}\right)^2$

$\displaystyle  \frac{1}{m(f_m)^2}=\left(2\pi f_m \frac{T_R}{13.8}\right)^2$

$\displaystyle \left( \frac{1}{m(f_m)^2}-1 \right)^{-\frac{1}{2}}=\left(2\pi f_m \frac{T_R}{13.8}\right)$

$\displaystyle \frac{\left( \frac{1}{m(f_m)^2}-1 \right)^{-\frac{1}{2}}}{(2\pi f_m)} =\frac{T_R}{13.8}$

$\displaystyle  T_R = \frac{\left( \frac{1}{m(f_m)^2}-1 \right)^{-\frac{1}{2}}}{2\pi f_m}13.8 $

Generalized RIR model (Unoki, 2017)

$\displaystyle m(f_m, T_R, b,\rm{SNR})={\left[ 1+\left(2\pi f_m \frac{T_R}{13.8}\right)^2 \right]}^{-{\frac{2b-1}{2}}}\left(\frac{1}{1+10^{-{\frac{\rm{SNR}}{10}}}}\right).$

In [2]:
TAE = np.load('TAE.npy')
MTF = np.load('MTF.npy')

In [5]:
MTF.shape

(431, 14, 7)

In [6]:
TAE.shape

(431, 6000, 7)

In [13]:
seed = 8
np.random.seed(seed)
epochs = 200
batch_size= 24

x_train = TAE
y_train = MTF

x_train = x_train.astype('float32') # gpus only work with float32
y_train = y_train.astype('float32')

x_train = np.reshape(x_train,[len(x_train),6000*7])
y_train = np.reshape(y_train,[len(y_train),14*7]) 

In [14]:
x_train.shape

(431, 42000)

## Feedforward Network

In [17]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(42000,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(98, activation='sigmoid'))

model.summary()

keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss='mse',optimizer='rmsprop',metrics=['mean_squared_error'])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               10752256  
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_10 (Dense)             (None, 98)                12642     
Total params: 10,797,794
Trainable params: 10,797,794
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
18/18 [==============================] - 1s 76ms/step - loss: 0.0694 - mean_squared_error: 0.0694
Epoch 2/200
18/18 [==============================] - 1s 76ms/step - loss: 0.0354 - mean_squared_error: 0.0354
Epoch 3/200
18/18 [==============================] - 1s 77ms/step - loss: 0.0331 - mean_squared_error: 0.0331
Epoch 4/200
18/18 [=======================

18/18 [==============================] - 1s 77ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 69/200
18/18 [==============================] - 1s 78ms/step - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 70/200
18/18 [==============================] - 1s 77ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 71/200
18/18 [==============================] - 1s 78ms/step - loss: 0.0028 - mean_squared_error: 0.0028
Epoch 72/200
18/18 [==============================] - 1s 79ms/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 73/200
18/18 [==============================] - 1s 79ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 74/200
18/18 [==============================] - 1s 77ms/step - loss: 0.0028 - mean_squared_error: 0.0028
Epoch 75/200
18/18 [==============================] - 1s 77ms/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 76/200
18/18 [==============================] - 1s 77ms/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 77/200
1

In [ ]:
model.save("modelFCN_july2020_1.h5")

## 1-Dimensional CNN

In [ ]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=100,strides=1, name='conv1D_Input', input_shape=(42000)))
model.add(Activation('relu'))
model.add(MaxPooling1D(2))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(98, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='mse',metrics=['mean_squared_error'])
model.summary()

keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss='mse',optimizer='rmsprop',metrics=['mean_squared_error'])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

In [2]:
pwd

'C:\\Users\\golf_\\estMTF_code'

# Evaluation with the baseline

In [3]:
model=load_model("modelFCN_july2020_1.h5")

In [8]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               10752256  
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_10 (Dense)             (None, 98)                12642     
Total params: 10,797,794
Trainable params: 10,797,794
Non-trainable params: 0
_________________________________________________________________


In [16]:
ls

 Volume in drive C is OS
 Volume Serial Number is 0AE9-9254

 Directory of C:\Users\golf_\estMTF_code

07/16/2020  04:05 PM    <DIR>          .
07/16/2020  04:05 PM    <DIR>          ..
07/16/2020  10:09 AM    <DIR>          .ipynb_checkpoints
07/03/2020  07:29 AM         1,418,368 a11_034_TAEs.mat
07/03/2020  07:51 AM               927 a11_MTF.mat
07/16/2020  04:05 PM            38,708 estMTF_PhD.ipynb
07/15/2020  06:46 PM        86,411,520 modelFCN_july2020_1.h5
07/03/2020  07:32 AM           338,032 MTF.npy
07/02/2020  11:02 AM       144,816,128 TAE.npy
               6 File(s)    233,023,683 bytes
               3 Dir(s)  830,887,051,264 bytes free


In [4]:
a11 = scipy.io.loadmat('a11_034_TAEs.mat')

In [6]:
a11 = scipy.io.loadmat('a11_MTF.mat')

In [5]:
TAE_a11 = a11['a11_034_TAEs']

In [9]:
a11_M = scipy.io.loadmat('a11_034_TAEs.mat')

In [10]:
MTF_a11 = a11['MTF']

In [12]:
print(MTF_a11.shape)
print(TAE_a11.shape)

(7, 14)
(7, 26729)


In [27]:
TAE_a11.shape

(7, 26729)

In [13]:
TAE_a11 = TAE_a11.transpose()

In [31]:
TAE_a11.shape

(26729, 7)

In [14]:
TAE_a11 = TAE_a11[0:6000,:]

In [15]:
TAE_a11_re =  np.reshape(TAE_a11,[1,6000*7])

In [16]:
estMTF_a11 = model.predict(TAE_a11_re)

In [39]:
estMTF_a11.shape

(1, 98)

In [18]:
a11_MTF_RIR = np.reshape(MTF_a11,[1,98])

In [37]:
a11_MTF_RIR.shape

(1, 98)

In [20]:
estMTF_a11 

array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 0.99923635, 0.999092  , 0.99860364,
        0.9986504 , 0.9986695 , 0.9984802 , 0.99920857, 0.99743253,
        0.99630105, 0.99701875, 0.9967468 , 0.99566096, 0.9961037 ,
        0.9984831 , 0.99539566, 0.99252695, 0.9924443 , 0.9917838 ,
        0.9915465 , 0.9931972 , 0.99629927, 0.98950106, 0.9825442 ,
        0.98240954, 0.9832519 , 0.98456895, 0.9863142 , 0.9931298 ,
        0.98322135, 0.976562  , 0.9724294 , 0.9670383 , 0.9718472 ,
        0.97714376, 0.987941  , 0.97259575, 0.95434594, 0.9532133 ,
        0.9481146 , 0.9506273 , 0.96504676, 0.9790315 , 0.946246  ,
        0.9255525 , 0.92164457, 0.9101263 , 0.9239783 , 0.9458584 ,
        0.9721784 , 0.8932365 , 0.8734469 , 0.86508214, 0.87455034,
        0.8789532 , 0.9238799 , 0.95306724, 0.8293685 , 0.82616764,
        0.78880405, 0.8406404 , 0.8223965 , 0.87447286, 0.9265329 ,
        0.7618637 , 0.74236053, 0.71994674, 0.80

In [31]:
RMSE = np.sqrt(mean_squared_error(a11_MTF_RIR,estMTF_a11 ))
print("%0.3f"%RMSE)

0.290


In [38]:
estMTF_a11.shape

(1, 98)

In [39]:
m =  np.reshape(estMTF_a11,[1,7,14])

In [40]:
m.shape

(1, 7, 14)

In [42]:
m

array([[[1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 0.99923635, 0.999092  , 0.99860364,
         0.9986504 , 0.9986695 , 0.9984802 , 0.99920857],
        [0.99743253, 0.99630105, 0.99701875, 0.9967468 , 0.99566096,
         0.9961037 , 0.9984831 , 0.99539566, 0.99252695, 0.9924443 ,
         0.9917838 , 0.9915465 , 0.9931972 , 0.99629927],
        [0.98950106, 0.9825442 , 0.98240954, 0.9832519 , 0.98456895,
         0.9863142 , 0.9931298 , 0.98322135, 0.976562  , 0.9724294 ,
         0.9670383 , 0.9718472 , 0.97714376, 0.987941  ],
        [0.97259575, 0.95434594, 0.9532133 , 0.9481146 , 0.9506273 ,
         0.96504676, 0.9790315 , 0.946246  , 0.9255525 , 0.92164457,
         0.9101263 , 0.9239783 , 0.9458584 , 0.9721784 ],
        [0.8932365 , 0.8734469 , 0.86508214, 0.87455034, 0.8789532 ,
         0.9238799 , 0.95306724, 0.8293685 , 0.82616764, 0.78880405,
         0.8406404 , 0.8223965 , 0.87447286, 0.9265329 ],
        [0.7618637 ,

In [41]:
mtf = np.zeros([7])

i=0
for i in range(7):
    mtf[i] = np.mean(m[:,i])
    print("%0.4f"%mtf[i])
    

0.9994
0.9951
0.9813
0.9478
0.8693
0.7549
0.6192


## Calculate RT60

In [30]:
fm = [0.63, 0.8, 1.0, 1.25, 1.6, 2.0, 2.5, 3.15, 4.0, 5.0, 6.3, 8.0, 10, 12.5]
RT = np.zeros([7,14])
c = 0
for i in range(6):
    for k in range(14):
        RT[i,k] = np.sqrt( (1/pow(mtf[i],2)) -1)  *(13.8/(2*np.pi*fm[k]))
    a = np.mean(RT[i,:])
    print("RT : %0.3f"%a)
    c = c+a
d = c/6    
print("final RT : %0.3f" %d) 

RT : 0.039
RT : 0.116
RT : 0.228
RT : 0.390
RT : 0.659
RT : 1.008
final RT : 0.407


In [32]:
a11_MTF_RIR.shape

(1, 98)

In [34]:
m_a11_RIR =  np.reshape(a11_MTF_RIR,[1,7,14])

In [35]:
mtf = np.zeros([7])

i=0
for i in range(7):
    mtf[i] = np.mean(m_a11_RIR[:,i])
    print("%0.4f"%mtf[i])
    

0.8121
0.7515
0.7831
0.7429
0.7718
0.7684
0.7665


In [36]:
fm = [0.63, 0.8, 1.0, 1.25, 1.6, 2.0, 2.5, 3.15, 4.0, 5.0, 6.3, 8.0, 10, 12.5]
RT = np.zeros([7,14])
c = 0
for i in range(6):
    for k in range(14):
        RT[i,k] = np.sqrt( (1/pow(mtf[i],2)) -1)  *(13.8/(2*np.pi*fm[k]))
    a = np.mean(RT[i,:])
    print("RT : %0.3f"%a)
    c = c+a
d = c/6    
print("final RT : %0.3f" %d) 

RT : 0.834
RT : 1.018
RT : 0.921
RT : 1.045
RT : 0.956
RT : 0.966
final RT : 0.957
